In [1]:
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd

First, let's get the wikipedia page of interest, then use BeautifulSoup to parse it

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [3]:
soup = BeautifulSoup(source, 'lxml')

let's open a csv file and write to it our headers

In [19]:
csv_file = open('postal_code.csv', 'w')

In [20]:
csv_writer = csv.writer(csv_file)

In [21]:
csv_writer.writerow(['PostalCode','Borough', 'Neighbourhood'])

34

In [22]:
table = soup.find('table')

In [23]:
for row in table.findAll('tr'):
    items = row.findAll('td')
    if len(items) == 3:
        csv_writer.writerow([items[0].text.strip(), items[1].text.strip(), items[2].text.strip()])

In [24]:
csv_file.close()

Close the file and open it with pandas

In [88]:
data = pd.read_csv('postal_code.csv')

In [89]:
data.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


If any cell doesn't have an assigned borough we  can drop it. If there are any borough's with no assigned neighbourhoods we can fix it

In [91]:
data.drop(data.loc[data.Borough == 'Not assigned'].index,inplace=True)
data.reset_index(drop=True,inplace=True)

In [92]:
data.loc[data.Neighbourhood == 'Not assigned']

,PostalCode,Borough,Neighbourhood
6,M7A,Queen's Park,Not assigned


In [93]:
data.at[6, 'Neighbourhood'] = "Queen's Park"

Let's find all the unique postal codes and join the neighbourhoods together

In [94]:
unique = list(data.PostalCode.unique())

In [95]:
new_data = {}
for postal in unique:
    this = data.loc[data.PostalCode == postal]
    neighbourhood = list(this.Neighbourhood.unique())
    Borough = list(this.Borough.unique())
    new_data[postal] = [''.join(Borough), ', '.join(neighbourhood)]

In [96]:
pd_2 = pd.DataFrame.from_dict(new_data, orient='index', columns=['Borough', 'Neighborhood'])
pd_2.reset_index(drop=False,inplace=True)

In [97]:
pd_2.head()

,index,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [100]:
pd_2.rename(columns={'index': 'PostalCode'},inplace=True)

In [105]:
pd_2.head(15)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [104]:
pd_2.shape

(103, 3)

All done

# Geospatial Data

In [110]:
geospatial = pd.read_csv('Geospatial_Coordinates.csv')

In [111]:
geospatial.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [119]:
combined = pd_2.set_index('PostalCode').join(geospatial.set_index('Postal Code'))
combined.reset_index(inplace=True)

In [120]:
combined.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [149]:
combined.shape

(103, 5)

# Clustering

In [123]:
import folium # map rendering library
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim

address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [124]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(combined['Latitude'], combined['Longitude'], combined['Borough'], combined['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [125]:
credintials = open('id.txt', 'r')
CLIENT_ID = credintials.readline() # your Foursquare ID
CLIENT_SECRET = credintials.readline() # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30

In [126]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [127]:
toronto_venues = getNearbyVenues(names=combined['Neighborhood'],
                                   latitudes=combined['Latitude'],
                                   longitudes=combined['Longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront, Regent Park
Lawrence Heights, Lawrence Manor
Queen's Park
Islington Avenue
Rouge, Malvern
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Highland Creek, Rouge Hill, Port Union
Flemingdon Park, Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Thorncliffe Park
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
East Birchmount Park, Ionview, Kennedy Park
Bayview Village
CFB Toronto, Downsview East
The D

In [128]:
print(toronto_venues.shape)
toronto_venues.head()

(1321, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,GTA Restoration,43.753396,-79.333477,Fireworks Store
3,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
4,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena


In [129]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",30,30,30,30,30,30
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3,3,3,3,3,3
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",11,11,11,11,11,11
"Alderwood, Long Branch",10,10,10,10,10,10
"Bathurst Manor, Downsview North, Wilson Heights",17,17,17,17,17,17
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24
Berczy Park,30,30,30,30,30,30


In [130]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [132]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.033333,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
1,Agincourt,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
4,"Alderwood, Long Branch",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
5,"Bathurst Manor, Downsview North, Wilson Heights",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.058824,0.000000,0.000000,0.0,0.000000,0.000000
6,Bayview Village,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
7,"Bedford Park, Lawrence Manor East",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.041667
8,Berczy Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
9,"Birch Cliff, Cliffside West",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000


In [142]:
#print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
              venue  freq
0        Steakhouse  0.10
1              Café  0.10
2             Hotel  0.07
3  Asian Restaurant  0.07
4  Greek Restaurant  0.03


----Agincourt----
            venue  freq
0  Clothing Store  0.25
1  Breakfast Spot  0.25
2    Skating Rink  0.25
3          Lounge  0.25
4     Opera House  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
             venue  freq
0   Cluster Labels  1.00
1             Park  0.33
2       Playground  0.33
3      Coffee Shop  0.33
4  Harbor / Marina  0.00


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                 venue  freq
0        Grocery Store  0.18
1  Fried Chicken Joint  0.09
2             Pharmacy  0.09
3  Japanese Restaurant  0.09
4       Discount Store  0.09


----Alderwood, Long Branch----
          venue  freq
0   Pizza Place   0.2
1          Bank   0.1
2  Skating Rink   0.1
3   Coffee Shop

               venue  freq
0     Cluster Labels  1.00
1  Convenience Store  0.33
2      Metro Station  0.33
3               Park  0.33
4             Office  0.00


----Emery, Humberlea----
                           venue  freq
0                 Cluster Labels   2.0
1  Paper / Office Supplies Store   0.5
2                 Baseball Field   0.5
3            Monument / Landmark   0.0
4                         Lounge   0.0


----Fairview, Henry Farm, Oriole----
                  venue  freq
0        Clothing Store  0.13
1           Coffee Shop  0.10
2         Smoothie Shop  0.03
3  Fast Food Restaurant  0.03
4                  Bank  0.03


----First Canadian Place, Underground city----
         venue  freq
0         Café  0.13
1  Coffee Shop  0.10
2   Steakhouse  0.07
3          Bar  0.07
4   Restaurant  0.07


----Flemingdon Park, Don Mills South----
                venue  freq
0    Asian Restaurant  0.10
1                 Gym  0.10
2          Beer Store  0.10
3         Coffee Shop  0.10


                 venue  freq
0          Coffee Shop  0.25
1                Trail  0.25
2                  Pub  0.25
3  Monument / Landmark  0.00
4         Liquor Store  0.00


----The Beaches West, India Bazaar----
                venue  freq
0                Park  0.11
1           Pet Store  0.06
2    Sushi Restaurant  0.06
3      Ice Cream Shop  0.06
4  Italian Restaurant  0.06


----The Danforth West, Riverdale----
                venue  freq
0    Greek Restaurant  0.30
1      Ice Cream Shop  0.07
2  Italian Restaurant  0.07
3     Bubble Tea Shop  0.03
4      Cosmetics Shop  0.03


----The Junction North, Runnymede----
         venue  freq
0  Pizza Place  0.33
1       Bakery  0.33
2     Bus Line  0.33
3  Yoga Studio  0.00
4        Motel  0.00


----The Kingsway, Montgomery Road, Old Mill North----
               venue  freq
0     Cluster Labels   1.0
1              River   0.5
2               Park   0.5
3        Opera House   0.0
4  Mobile Phone Shop   0.0


----Thorncliffe Park----

In [143]:
#put above into data frame
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [144]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Steakhouse,Café,Asian Restaurant,Hotel,Speakeasy,Bar,Smoke Shop,Seafood Restaurant,Plaza,Pizza Place
1,Agincourt,Clothing Store,Skating Rink,Lounge,Breakfast Spot,Cluster Labels,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Cluster Labels,Playground,Coffee Shop,Park,Airport,Empanada Restaurant,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Beer Store,Fast Food Restaurant,Coffee Shop,Discount Store,Sandwich Place,Pharmacy,Pizza Place,Japanese Restaurant,Fried Chicken Joint
4,"Alderwood, Long Branch",Pizza Place,Pool,Sandwich Place,Skating Rink,Bank,Pharmacy,Pub,Gym,Coffee Shop,Dim Sum Restaurant
5,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Shopping Mall,Fried Chicken Joint,Sushi Restaurant,Bridal Shop,Middle Eastern Restaurant,Pizza Place,Bank,Pharmacy,Restaurant
6,Bayview Village,Chinese Restaurant,Bank,Japanese Restaurant,Café,Cluster Labels,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run
7,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Fast Food Restaurant,Sushi Restaurant,Comfort Food Restaurant,Café,Pharmacy,Butcher,Pizza Place,Pub
8,Berczy Park,Cocktail Bar,Café,Bakery,Seafood Restaurant,Farmers Market,Park,Concert Hall,Liquor Store,Pub,Steakhouse
9,"Birch Cliff, Cliffside West",Skating Rink,General Entertainment,College Stadium,Café,Cluster Labels,Curling Ice,Dumpling Restaurant,Drugstore,Dog Run,Discount Store


In [155]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 3, 0, 0, 0, 0, 0, 0, 0])

In [157]:
toronto_merged = neighborhoods_venues_sorted

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(combined.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,PostalCode,Borough,Latitude,Longitude
0,"Adelaide, King, Richmond",Steakhouse,Café,Asian Restaurant,Hotel,Speakeasy,Bar,Smoke Shop,Seafood Restaurant,Plaza,Pizza Place,0,M5H,Downtown Toronto,43.650571,-79.384568
1,Agincourt,Clothing Store,Skating Rink,Lounge,Breakfast Spot,Cluster Labels,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner,0,M1S,Scarborough,43.794200,-79.262029
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Cluster Labels,Playground,Coffee Shop,Park,Airport,Empanada Restaurant,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,3,M1V,Scarborough,43.815252,-79.284577
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Beer Store,Fast Food Restaurant,Coffee Shop,Discount Store,Sandwich Place,Pharmacy,Pizza Place,Japanese Restaurant,Fried Chicken Joint,0,M9V,Etobicoke,43.739416,-79.588437
4,"Alderwood, Long Branch",Pizza Place,Pool,Sandwich Place,Skating Rink,Bank,Pharmacy,Pub,Gym,Coffee Shop,Dim Sum Restaurant,0,M8W,Etobicoke,43.602414,-79.543484


In [158]:
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters